# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [75]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [76]:
# Import weather file as DataFrame
cities_pd = pd.read_csv("../output_data/cities.csv")
cities_pd.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [77]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [ ]:
# Store 'Lat' and 'Lng' into  locations and convert it to float
locations = cities_pd[["Lat", "Lng"]].astype(float)

#Store Humidity into weights
weights = cities_pd[["Humidity"]].astype(float)

# produce a heatmap layer
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100,
                                 point_radius = 1))
fig

In [ ]:
# Use the Lat and Lng as locations and Humididty as the weight
#Quotation marks is use to specify initial entries.
cities_pd["Lat"] = ""
cities_pd["Lng"] = ""
cities_pd["Humidity"] = ""
cities_pd.head(10)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [79]:
narrowed_city = cities_pd.loc[(cities_pd["Wind Speed"] <= 10) & (cities_pd["Cloudiness"] == 0) & \
                                   (cities_pd["Max Temp"] >= 70) & (cities_pd["Max Temp"] <= 80)].dropna()

narrowed_city

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#Store Hotel Map into variable hotel_df and add it to the dataframe
hotel_df = []

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in cities_pd.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: for each city's coordinates
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    try:
        hotel_df.append(jsn['results'][0]['name'])
    except:
        hotel_df.append("")

narrowed_city["Hotel Name"] = hotel_df
narrowed_city_df = narrowed_city.dropna(how='any')
narrowed_city_df.head(12)



In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig